<a href="https://colab.research.google.com/github/yaakitori/negima/blob/main/gat_MUTAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GATの実装
Graph Attention Networks（GAT）を実装
多数のグラフを持つデータセットを訓練データに使い、ミニバッチ法により学習を行う


In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


Googleドライブ上のパスを指定

In [ ]:
dir_name = "Library/GNN"  # 好きなパスを設定してください
package_path = "/content/drive/MyDrive/" + dir_name + "/packages/"

## PyTorch Geometricのインストール


In [ ]:
# !pip install --no-cache-dir torch-geometric torch-sparse torch-scatter -t $package_path

Google Driveに保存したパッケージをシステムに追加  

In [ ]:
import sys

sys.path.append(package_path)

## データセットの読み込み
TUDatasetから、188のグラフが含まれるデータセット「MUTAG」を読み込む


In [ ]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

dataset = TUDataset(root="/tmp/MUTAG", name="MUTAG")

dataset = dataset.shuffle()  # データセットをシャッフル
dataset_train = dataset[:140]  # 訓練用データセット
dataset_test = dataset[140:]  # テスト用データセット

batch_size = 64  # バッチサイズ
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

Processing...
Done!


## モデルの構築
GAT層の実装には、`GATConv()`を利用
```
GATConv(入力の特徴量数, 出力の特徴量数, Multi-head Attentionの数)
```  
https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.conv.GATConv  
GATConv層を2つ重ね、全結合層の前にニューロンをランダムに削除する「ドロップアウト」を導入

In [ ]:
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv
from torch_geometric.nn import global_mean_pool

n_h = 64  # 中間層における特徴量の数
n_head = 32

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class GAT(nn.Module):
    def __init__(self):
        super().__init__()
        self.gat1 = GATConv(dataset.num_node_features,
                            n_h,
                            heads=n_head) # 特に設定しなければ各ヘッドの出力は結合される
        self.gat2 = GATConv(n_h*n_head, # 1層目のGATConvの出力の数（中間層における特徴量の数×ヘッドの数）
                            n_h,
                            heads=n_head)
        self.fc = nn.Linear(n_h*n_head, dataset.num_classes)  # 全結合層

        self.relu = nn.ReLU()  # ReLU
        self.dropout = nn.Dropout(p=0.5)  # ドロップアウト:(p=ドロップアウト率)

    def forward(self, data):
        x = data.x
        edge_index = data.edge_index
        batch = data.batch

        x = self.gat1(x, edge_index)
        x = self.relu(x)
        x = self.gat2(x, edge_index)
        x = self.relu(x)

        # 全てのノードで各特徴量の平均をとる
        # ノードの次元をつぶす＋ヘッドの数分、各ノードに特徴量があるのでそれも含め平均とる
        x = global_mean_pool(x, batch)  # (バッチサイズ, 特徴量の数)に変換

        x = self.dropout(x)
        x = self.fc(x)

        return x

net = GAT()
net.to(device)

GAT(
  (gat1): GATConv(7, 64, heads=32)
  (gat2): GATConv(2048, 64, heads=32)
  (fc): Linear(in_features=2048, out_features=2, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.5, inplace=False)
)

## 学習

モデル評価用の関数を用意

In [ ]:
def eval(loader):
    correct = 0  # 正解数

    for data in loader:
        data = data.to(device)
        out = net(data)
        pred = out.argmax(dim=1)
        correct += int((pred == data.y).sum())

    return correct/len(loader.dataset)  # 正解率

訓練データを使い、モデルを訓練

In [ ]:
from torch import optim

# 交差エントロピー誤差関数
loss_fnc = nn.CrossEntropyLoss()

# 最適化アルゴリズム
optimizer = optim.Adam(net.parameters())

for epoch in range(200):
    # 訓練
    net.train()  # 訓練モード
    for data in loader_train:
        data = data.to(device)

        optimizer.zero_grad()  # 勾配の初期化
        out = net(data)  # 順伝播により予測値を得る
        loss = loss_fnc(out, data.y)  # 予測値と正解値から誤差を計算

        loss.backward()  # 誤差からバックプロパゲーションにより勾配を計算
        optimizer.step()  # 最適化アルゴリズムによりパラメータを更新

    # 評価
    net.eval()  # 評価モード
    acc_train = eval(loader_train)
    acc_test = eval(loader_test)
    print("Epoch:", epoch,
          "acc_train:", str(acc_train*100) + "%",
          "acc_test:", str(acc_test*100) + "%")

Epoch: 0 acc_train: 63.57142857142857% acc_test: 75.0%
Epoch: 1 acc_train: 63.57142857142857% acc_test: 75.0%
Epoch: 2 acc_train: 63.57142857142857% acc_test: 75.0%
Epoch: 3 acc_train: 63.57142857142857% acc_test: 75.0%
Epoch: 4 acc_train: 63.57142857142857% acc_test: 75.0%
Epoch: 5 acc_train: 63.57142857142857% acc_test: 75.0%
Epoch: 6 acc_train: 63.57142857142857% acc_test: 75.0%
Epoch: 7 acc_train: 65.0% acc_test: 75.0%
Epoch: 8 acc_train: 66.42857142857143% acc_test: 79.16666666666666%
Epoch: 9 acc_train: 71.42857142857143% acc_test: 83.33333333333334%
Epoch: 10 acc_train: 71.42857142857143% acc_test: 85.41666666666666%
Epoch: 11 acc_train: 70.71428571428572% acc_test: 81.25%
Epoch: 12 acc_train: 71.42857142857143% acc_test: 83.33333333333334%
Epoch: 13 acc_train: 71.42857142857143% acc_test: 79.16666666666666%
Epoch: 14 acc_train: 68.57142857142857% acc_test: 81.25%
Epoch: 15 acc_train: 71.42857142857143% acc_test: 83.33333333333334%
Epoch: 16 acc_train: 73.57142857142858% acc_tes

## モデルの評価
訓練済みのモデルを評価

In [ ]:
net.eval()  # 評価モード
acc_test = eval(loader_test)
print("accuracy:", str(acc_test*100) + "%")

accuracy: 83.33333333333334%
